In [ ]:
onnx_model_path = "./model/model.onnx"
ExecutionProvider="DmlExecutionProvider"

In [ ]:
def _fix_winrt_runtime():
    """This function removes the msvcp140.dll from the winrt-runtime package.
    So it does not cause issues with other libraries.
    """
    from importlib import metadata
    from pathlib import Path
    site_packages_path = Path(str(metadata.distribution('winrt-runtime').locate_file('')))
    dll_path = site_packages_path / 'winrt' / 'msvcp140.dll'
    if dll_path.exists():
        dll_path.unlink()
            
def _get_ep_paths() -> dict[str, str]:
    from winui3.microsoft.windows.applicationmodel.dynamicdependency.bootstrap import (
        InitializeOptions,
        initialize
    )
    import winui3.microsoft.windows.ai.machinelearning as winml
    eps = {}
    with initialize(options = InitializeOptions.ON_NO_MATCH_SHOW_UI):
        catalog = winml.ExecutionProviderCatalog.get_default()
        providers = catalog.find_all_providers()
        for provider in providers:
            provider.ensure_ready_async().get()
            eps[provider.name] = provider.library_path
            # DO NOT call provider.try_register in python. That will register to the native env.
    return eps

def _regsiter_executino_providers_to_onnxruntime():
    import onnxruntime as ort

    paths = _get_ep_paths()
    for item in paths.items():
        print(f"----register ort ep---- {item[0]} {item[1]}")
        ort.register_execution_provider_library(item[0], item[1])

_fix_winrt_runtime()
_regsiter_executino_providers_to_onnxruntime()

In [ ]:
import requests
from PIL import Image
url = "https://onnxruntime.ai/images/dog.jpeg"
response = requests.get(url)
# Save the image to a file
with open("dog.jpeg", "wb") as file:
    file.write(response.content)
img = Image.open("dog.jpeg")
img

In [ ]:
import onnxruntime as ort
from PIL import Image
import torch
import torchvision.transforms as transforms
from torchvision.models.resnet import ResNet50_Weights
import numpy as np

image_file_path = "dog.jpeg"

# Create ONNX runtime session
def add_ep_for_device(session_options, ep_name, device_type, ep_options=None):
    ep_devices = ort.get_ep_devices()
    for ep_device in ep_devices:
        if ep_device.ep_name == ep_name and ep_device.device.type == device_type:
            print(f"Adding {ep_name} for {device_type}")
            session_options.add_provider_for_devices([ep_device], {} if ep_options is None else ep_options)


session_options = ort.SessionOptions()

add_ep_for_device(session_options, ExecutionProvider, ort.OrtHardwareDeviceType.NPU)

session = ort.InferenceSession(
    onnx_model_path, # a model with QNN EPContext nodes
    sess_options=session_options,
)

print("Available providers:", session.get_providers())
print("Current provider:", session.get_provider_options())

# Read and preprocess image
image = Image.open(image_file_path)
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
input_tensor = preprocess(image)
input_batch = input_tensor.unsqueeze(0)

# Run inference
ort_inputs = {session.get_inputs()[0].name: input_batch.numpy().astype(np.float16)}
ort_outputs = session.run(None, ort_inputs)

# Postprocess to get softmax vector
output = ort_outputs[0]
softmax = torch.nn.functional.softmax(torch.tensor(output), dim=1)

# Extract top 10 predicted classes
top10 = torch.topk(softmax, 10)

# Get label mapping
weights = ResNet50_Weights.DEFAULT
labels = weights.meta["categories"]

# Print results to console
print("Top 10 predictions for ResNet50 v2...")
print("--------------------------------------------------------------")
for i in range(10):
    print(f"Label: {labels[top10.indices[0][i]]}, Confidence: {top10.values[0][i].item():.4f}")
